In [1]:
from IPython.display import HTML, Javascript, clear_output
import os
import sys

from google.colab import drive

drive.mount("/content/drive")

!pip install human_id

# Define the repo URL and folder name
REPO_URL = "https://github.com/rosinusserrano/autoencoding_experiments.git"
LOCAL_DIR = "/content/autoencoding_experiments"

# Inject a floating button with JavaScript
def inject_floating_button():
    display(HTML("""
    <script>
    function reloadRepo() {
        google.colab.kernel.invokeFunction('notebook.reload_repo', [], {});
    }
    const button = document.createElement('button');
    button.innerHTML = 'Reload Repo';
    button.style.position = 'fixed';
    button.style.bottom = '20px';
    button.style.right = '20px';
    button.style.backgroundColor = '#ff5050';
    button.style.color = 'white';
    button.style.border = 'none';
    button.style.padding = '10px 20px';
    button.style.borderRadius = '5px';
    button.style.boxShadow = '0px 4px 6px rgba(0,0,0,0.1)';
    button.style.cursor = 'pointer';
    button.onclick = reloadRepo;
    document.body.appendChild(button);
    </script>
    """))

def clone_and_import():
    # Clear output and display new button
    clear_output(wait=True)
    inject_floating_button()
    # Clone the repository if it doesn't exist
    if not os.path.exists(LOCAL_DIR):
        !git clone {REPO_URL} {LOCAL_DIR}
    else:
        # Pull the latest changes if the repo already exists
        !cd {LOCAL_DIR} && git pull

    # Append the repository to the system path
    if LOCAL_DIR not in sys.path:
        sys.path.append(LOCAL_DIR + "/src")
    print(f"Repository at '{LOCAL_DIR}' is ready for imports.")

# Register the Python function as a callable function for JavaScript
from google.colab import output
output.register_callback('notebook.reload_repo', clone_and_import)

# Inject the button
inject_floating_button()

# Initial setup
clone_and_import()

Cloning into '/content/autoencoding_experiments'...
remote: Enumerating objects: 341, done.
remote: Counting objects: 100% (341/341), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 341 (delta 192), reused 263 (delta 121), pack-reused 0 (from 0)
Receiving objects: 100% (341/341), 112.57 KiB | 4.17 MiB/s, done.
Resolving deltas: 100% (192/192), done.
Repository at '/content/autoencoding_experiments' is ready for imports.


In [2]:

%reload_ext autoreload
%autoreload 2

In [ ]:
from functools import partial

from datasets import DatasetConfig
from logger.caveman import CavemanLogger
from models.vae import VAEConfig, mse_and_kld_loss
from utils.train import TrainConfig, standard_training_pipeline

model_config = VAEConfig()
dataset_config = DatasetConfig(
    dataset_name="stl10",
    validation_split=0.1,
    batch_size=128,
)
train_config = TrainConfig("adam", learning_rate=0.0003, n_epochs=50)

experiment_name = "VAE on STL10"
# logger = CavemanLogger(f"../runs/{experiment_name}")
logger = CavemanLogger(f"/content/drive/MyDrive/Autoencoding Experiments/{experiment_name}")

standard_training_pipeline(
    model_config=model_config,
    dataset_config=dataset_config,
    train_config=train_config,
    logger=logger,
    loss_fn=partial(mse_and_kld_loss, model_config=model_config),
    validation_interval=1,
    test_interval=10,
    visualization_interval=1,
)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


100%|██████████| 2.64G/2.64G [03:49<00:00, 11.5MB/s]


Extracting data/stl10_binary.tar.gz to data
Files already downloaded and verified
On device: cpu


  0%|          | 0/704 [00:00<?, ?it/s]